# Métodos iterativos creados a mano

In [72]:
using LinearAlgebra, SparseArrays, BenchmarkTools

In [27]:
function get_A_b(N)
    dv = fill(3., (N,))
    ev = fill(-1., (N-1,))
    A = Tridiagonal(ev,dv, ev)
    A = sparse(A)
    b = fill(1.5, (N,))
    b[1] = b[end] = 5/2
    isodd(N) ? b[ceil(Int, N/2)] = 1 : b[trunc(Int, N/2)] = b[trunc(Int, N/2+1)] = 1
    for row ∈ 1:ceil(Int,N/2)
        if isodd(N)
            if row != ceil(N/2)
                A[row,end-row+1] = A[end-row+1, row] = 0.5
            end
        else
            if (row != N/2) & (row != (N/2 + 1))
                A[row,end-row+1] = A[end-row+1, row] = 0.5
            end
        end
    end
    return A, b
end

get_A_b (generic function with 1 method)

## Jacobi

In [61]:
function jacobi(A, b, ϵ = 1e-5)
    n = size(A)[1]
    x = b
    x_bar = zeros(n)

    counter = 0
    while norm(A*x - b) > ϵ
        for i = 1:n
            x_bar[i] = 0
            for j = 1:n
                if j != i
                    x_bar[i] = x_bar[i] + A[i,j]*x[j]
                end
            end
            x_bar[i] = (b[i] - x_bar[i])/A[i,i]
        end
        x = x_bar
        counter+=1
    end
    println("Se necesitaron $counter iteraciones para obtener la solución con ϵ = $ϵ")
    return x
end

jacobi (generic function with 2 methods)

In [76]:
A, b = get_A_b(500)
t = @benchmark jacobi(A,b, 1e-11)

Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 75 iteraciones para obtener la solución con ϵ = 1.0e-11


BenchmarkTools.Trial: 60 samples with 1 evaluation.
 Range (min … max):  80.830 ms … 95.382 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     82.993 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   83.848 ms ±  2.846 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁ █▁  ▁                                                    
  ▆▄█▄██▇▆█▆▆▆▇▇▄▆▄▄▁▁▄▁▇▄▄▄▁▆▁▄▄▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  80.8 ms         Histogram: frequency by time        94.1 ms <

 Memory estimate: 623.12 KiB, allocs estimate: 190.

## Jacobi GPU

In [ ]:
function jacobi_gpu(iter=50,dim=20000)
    m = dim;
    A = sprandn(Float32,m,m,0.01) + Float32(m/100)*sparse(Matrix(1f0I, m, m));
    b = CUDA.ones(Float32,m);
    dA = diag(A);
    dA = CuArray(dA)
    N = - triu(A,1) - tril(A,-1);
    A = CuSparseMatrixCSR(A)
    N = CuSparseMatrixCSR(N)
    x = CUDA.zeros(Float32, size(b))
    normres = [];
    for k = 1:iter
        x = (N*x + b)./dA
        normres = [normres;norm(b-A*x)]
    end
    return normres
end

## Gauss-Seidel

In [64]:
function gauss_seidel(A, b, ϵ = 1e-5)
    n = size(A)[1]
    x = b
    
    counter = 0
    while norm(A*x - b) > ϵ
        x_next = zeros(n)
        for i = 1:n
            σ = 0
            for j = 1:i-1
                if j <= i
                    σ = σ + A[i,j]*x_next[j]
                end
            end
            for j = i+1:n
                σ = σ + A[i,j]*x[j]
            end
            x_next[i] = (b[i] - σ)/A[i,i]
        end
        x = x_next
        counter+=1
    end
    println("Se necesitaron $counter iteraciones para obtener la solución con ϵ = $ϵ")
    return x
end 

gauss_seidel (generic function with 2 methods)

In [75]:
A, b = get_A_b(500)
t = @benchmark gauss_seidel(A,b,1e-11)

Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 76 iteraciones para obtener la solución con ϵ = 1.0e-11


BenchmarkTools.Trial: 61 samples with 1 evaluation.
 Range (min … max):  78.946 ms … 102.018 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     81.584 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   82.867 ms ±   4.410 ms  ┊ GC (mean ± σ):  0.05% ± 0.39%

  ▆█ ▄▂   ▂                                                     
  ██▄██▆▁███▄█▁▆▆▁▄▄▄▄▁▄▆▄▄█▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▄ ▁
  78.9 ms         Histogram: frequency by time         96.9 ms <

 Memory estimate: 935.94 KiB, allocs estimate: 267.

## Gauss-Seidel GPU

## Gradiente conjugado

In [82]:
# Algoritmo sacado directo del pseudocódigo
function gradiente_conjugado(A,b,x,r,ρ,res,p,ϵ)
    counter = 0
    while  norm(res) > ϵ
        # Aquí iría la condición más chida de precondicionamiento
        ρ[1] = ρ[2] # Utilizamos sólo dos actualizaciones hacia atrás, por eso esto sí es una lista.
        ρ[2] = r'*r # Aquí cambia cuando es precondicionada
        if counter == 0
            p = r
        else
            β = ρ[2]/ρ[1]
            p = r + β*p
        end
        q = A*p
        α = ρ[2]/(p'*q)
        x = x + α*p
        r = r - α*q
        res = b - A*x
        counter+=1
    end
    println("Se necesitaron $counter iteraciones para obtener la solución con ϵ = $ϵ")
    return x
end

gradiente_conjugado (generic function with 1 method)

In [84]:
n = 500
A, b = get_A_b(n)
x = rand(n)
r = b - A*x
ρ = [0, r'*r]
res = b - A*x
p = Array{Float64}(undef, size(r,1), 1);
ϵ = 1e-11
t = @benchmark gradiente_conjugado(A,b,x,r,ρ,res,p,ϵ)

Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

...

Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

Excessive output truncated after 524359 bytes.

Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se necesitaron 44 iteraciones para obtener la solución con ϵ = 1.0e-11
Se nec

BenchmarkTools.Trial: 9092 samples with 1 evaluation.
 Range (min … max):  381.439 μs …   3.691 ms  ┊ GC (min … max):  0.00% … 82.48%
 Time  (median):     425.518 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   543.849 μs ± 388.502 μs  ┊ GC (mean ± σ):  12.90% ± 14.66%

  ▇█▅▅▄▃▃▄▄▂▂                                             ▂▁    ▂
  ████████████▇▆▆▆▆▄▃▅▄▃▁▃▁▁▃▁▃▁▄▃▃▁▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄████▇▇ █
  381 μs        Histogram: log(frequency) by time       2.52 ms <

 Memory estimate: 1.57 MiB, allocs estimate: 915.

## Gradiente conjugado GPU

---